<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Ignacio Meza, Gabriel Iturra
- Auxiliar: Sebastián Tinoco
- Ayudante: Arturo Lazcano, Angelo Muñoz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Michelle Avendaño
- Nombre de alumno 2: Claudia Navarro


## Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer material del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

### **Link de repositorio de GitHub:** `http://....`

# Importamos librerias útiles

In [ ]:
!pip install -qq xgboost optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 11.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, MinMaxScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import GradientBoostingRegressor
import joblib
from xgboost import XGBRegressor
import warnings
from tabulate import tabulate
import optuna
from optuna.samplers import TPESampler
from sklearn.model_selection import cross_val_score
from optuna.integration import XGBoostPruningCallback
import xgboost as xgb

# 1. El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Como usted tuvo un rendimiento sobresaliente en el proyecto de caracterización de datos, Fiu lo contrata como *data scientist* de su emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [ ]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except:
    print('Ignorando conexión drive-colab')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/sales.csv')
df

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696
...,...,...,...,...,...,...,...,...,...,...,...,...
7451,7555,31/12/18,Athens,37.97945,23.71622,664046,shop_1,kinder-cola,plastic,1.5lt,2.52,13760
7452,7556,31/12/18,Athens,37.97945,23.71622,664046,shop_1,orange-power,plastic,1.5lt,2.18,16309
7453,7557,31/12/18,Patra,38.24444,21.73444,168034,shop_6,kinder-cola,can,330ml,0.85,24378
7454,7558,31/12/18,Thessaloniki,40.64361,22.93086,354290,shop_4,adult-cola,plastic,1.5lt,2.17,20691


In [ ]:
df['date'] = pd.to_datetime(df['date'])
df.head()

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


## 1.1 Generando un Baseline (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad.
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas.
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas.
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios.
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio?
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`?
7. Guarde ambos modelos en un archivo .pkl (uno cada uno)

In [ ]:
df

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696
...,...,...,...,...,...,...,...,...,...,...,...,...
7451,7555,2018-12-31,Athens,37.97945,23.71622,664046,shop_1,kinder-cola,plastic,1.5lt,2.52,13760
7452,7556,2018-12-31,Athens,37.97945,23.71622,664046,shop_1,orange-power,plastic,1.5lt,2.18,16309
7453,7557,2018-12-31,Patra,38.24444,21.73444,168034,shop_6,kinder-cola,can,330ml,0.85,24378
7454,7558,2018-12-31,Thessaloniki,40.64361,22.93086,354290,shop_4,adult-cola,plastic,1.5lt,2.17,20691


Se analiza la variedad que presenta cada columna

In [ ]:
non_numeric_columns = df.select_dtypes(exclude=['number']).columns
for column in non_numeric_columns:
    print(f"Conteo de valores en '{column}':")
    print(tabulate(df[column].value_counts().reset_index(), headers=['Valor', 'Conteo'], tablefmt='pretty'))

Conteo de valores en 'date':
+----+---------------------+--------+
|    |        Valor        | Conteo |
+----+---------------------+--------+
| 0  | 2018-12-31 00:00:00 |   90   |
| 1  | 2016-01-31 00:00:00 |   90   |
| 2  | 2014-11-30 00:00:00 |   90   |
| 3  | 2018-02-28 00:00:00 |   90   |
| 4  | 2014-08-31 00:00:00 |   90   |
| 5  | 2014-07-31 00:00:00 |   90   |
| 6  | 2017-11-30 00:00:00 |   90   |
| 7  | 2018-03-31 00:00:00 |   90   |
| 8  | 2018-04-30 00:00:00 |   90   |
| 9  | 2018-05-31 00:00:00 |   90   |
| 10 | 2013-11-30 00:00:00 |   90   |
| 11 | 2018-06-30 00:00:00 |   90   |
| 12 | 2015-11-30 00:00:00 |   90   |
| 13 | 2018-01-31 00:00:00 |   90   |
| 14 | 2018-07-31 00:00:00 |   90   |
| 15 | 2014-12-31 00:00:00 |   90   |
| 16 | 2018-08-31 00:00:00 |   90   |
| 17 | 2016-03-31 00:00:00 |   90   |
| 18 | 2018-11-30 00:00:00 |   90   |
| 19 | 2016-09-30 00:00:00 |   90   |
| 20 | 2012-06-30 00:00:00 |   90   |
| 21 | 2018-10-31 00:00:00 |   90   |
| 22 | 2012-08-31 00:

A partir de lo anterior se decide que la varible capacity tenga una unica unidad de medida, asi que se transformar los litros a ml

In [ ]:

#Se tienen dos unidades para capacity, se transforman todas a ml
unique_values = df['capacity'].unique()
print("Los diferentes valores de 'capacity' son:")
print(unique_values)
df['capacity'] = df['capacity'].str.replace('.', '')  # Convertir litros a mililitros
df['capacity'] = df['capacity'].str.replace('lt', '00ml')  # Convertir litros a mililitros
unique_values = df['capacity'].unique()
print("Los diferentes valores nuevos de 'capacity' son:")
print(unique_values)

Los diferentes valores de 'capacity' son:
['500ml' '1.5lt' '330ml']
Los diferentes valores nuevos de 'capacity' son:
['500ml' '1500ml' '330ml']


<ipython-input-9-ed3430b0083a>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['capacity'] = df['capacity'].str.replace('.', '')  # Convertir litros a mililitros


Función para extraer el día, mes y año de la variable 'date'

In [ ]:
def extract_date_info(df):
    df_copy = df.copy()  # Crear una copia del DataFrame para no modificar el original
    df_copy['day'] = df_copy['date'].dt.day.astype('category')
    df_copy['month'] = df_copy['date'].dt.month.astype('category')
    df_copy['year'] = df_copy['date'].dt.year.astype('category')
    return df_copy.drop(columns=['date'])  # Eliminar la columna 'date'

# Aplicar la función para extraer la información de fecha al DataFrame
df = extract_date_info(df)

Se decide excluir las variables 'ID' (por carecer de información relevante), 'lat' y 'long' (ya que contamos con la variable 'city').

In [ ]:
columns_to_drop = ['id', 'lat', 'long', 'container']
df = df.drop(columns=columns_to_drop)
non_numeric_columns = df.select_dtypes(exclude=['number']).columns
for column in non_numeric_columns:
    print(f"Conteo de valores en '{column}':")
    print(tabulate(df[column].value_counts().reset_index(), headers=['Valor', 'Conteo'], tablefmt='pretty'))

Conteo de valores en 'city':
+---+--------------+--------+
|   |    Valor     | Conteo |
+---+--------------+--------+
| 0 |    Athens    |  2482  |
| 1 | Thessaloniki |  1246  |
| 2 |    Patra     |  1245  |
| 3 |    Larisa    |  1242  |
| 4 |  Irakleion   |  1241  |
+---+--------------+--------+
Conteo de valores en 'shop':
+---+--------+--------+
|   | Valor  | Conteo |
+---+--------+--------+
| 0 | shop_4 |  1246  |
| 1 | shop_6 |  1245  |
| 2 | shop_5 |  1242  |
| 3 | shop_1 |  1241  |
| 4 | shop_2 |  1241  |
| 5 | shop_3 |  1241  |
+---+--------+--------+
Conteo de valores en 'brand':
+---+--------------+--------+
|   |    Valor     | Conteo |
+---+--------------+--------+
| 0 | kinder-cola  |  1495  |
| 1 |  adult-cola  |  1493  |
| 2 | lemon-boost  |  1493  |
| 3 |    gazoza    |  1491  |
| 4 | orange-power |  1484  |
+---+--------------+--------+
Conteo de valores en 'capacity':
+---+--------+--------+
|   | Valor  | Conteo |
+---+--------+--------+
| 0 | 330ml  |  2486  |
| 1

In [ ]:
# Dividir los datos en conjuntos de entrenamiento, validación y prueba
X = df.drop(columns=['quantity'])
y = df['quantity']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=(2/9), random_state=42)

# Definir transformadores para columnas numéricas y categóricas
numeric_transformer = MinMaxScaler()
categorical_transformer = OneHotEncoder(drop='first', sparse=False)

# Definir preprocesamiento con ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('numeric', numeric_transformer, [ 'pop', 'price']),
        ('categorical', categorical_transformer, ['year','day', 'month', 'city', 'shop','brand','capacity'])
    ],
    remainder='passthrough'
)

# Definir el regresor Dummy con promedioos
dummy_regressor = DummyRegressor(strategy='mean')

# Crear el pipeline con pasos de transformación y el regresor
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),  # Preprocesamiento de datos
    ('dummy_regressor', dummy_regressor)  # Regresor Dummy, no sería necesario escalar la variable de salida, ya que el DummyRegressor no se ve afectado por la escala de la variable de salida.
])

# Entrenar el pipeline con los datos de entrenamiento
pipeline.fit(X_train, y_train)

# Realizar predicciones en el conjunto de validación
predictions = pipeline.predict(X_val)

# Evaluar métrica: Mean Absolute Error en el conjunto de validación
mae = mean_absolute_error(y_val, predictions)
print("Mean Absolute Error en set de validacion para dummy_regressor:", mae)

# Definir el nuevo modelo XGBRegressor
xgb_regressor = XGBRegressor()

# Crea un nuevo pipeline con el XGBRegressor
xgb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('xgb_regressor', xgb_regressor)
])

# Entrena el nuevo pipeline con XGBRegressor
xgb_pipeline.fit(X_train, y_train)

# Realiza predicciones en el conjunto de validación usando XGBRegressor
xgb_predictions_val = xgb_pipeline.predict(X_val)

# Evalúa la métrica: Mean Absolute Error en el conjunto de validación para XGBRegressor
xgb_mae_val = mean_absolute_error(y_val, xgb_predictions_val)
print("Mean Absolute Error en set de validacion para XGBRegressor:", xgb_mae_val)

# Guarda los modelos en archivos .pkl
joblib.dump(pipeline, 'dummy_regressor_model.pkl')
joblib.dump(xgb_pipeline, 'xgb_regressor_model.pkl')

Mean Absolute Error en set de validacion para dummy_regressor: 13543.961387782238


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Mean Absolute Error en set de validacion para XGBRegressor: 2518.8284681386667


['xgb_regressor_model.pkl']

El modelo XGBRegressor supera significativamente al dummy_regressor en la precisión de las predicciones. El XGBRegressor con un MAE de alrededor de 2518.82 es considerablemente más preciso que el dummy_regressor, que tiene un MAE de alrededor de 13543.96. Esto sugiere que el XGBRegressor es mucho más efectivo para predecir la demanda basada en la ciudad, precio, marca, tienda y demas variables seleccionadas.

## 1.2 Forzando relaciones entre parámetros con XGBoost (1.0 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo.

Vuelva a entrenar el `Pipeline`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo?

Nuevamente, guarde su modelo en un archivo .pkl

Nota: Para realizar esta parte, debe apoyarse en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>.

Hint: Para implementar el constraint, se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

In [ ]:
column_order =X_train.columns.tolist()
print(column_order)
# Definir el nuevo modelo XGBRegressor con relación monótona negativa
xgb_regressor_monotonic = XGBRegressor(monotone_constraints='(0, 0, 0, 0, 0, -1, 0, 0, 0)') #Según yo si esta bien :)

# Crear un nuevo pipeline con el XGBRegressor con relación monótona
xgb_pipeline_monotonic = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('xgb_regressor_monotonic', xgb_regressor_monotonic)
])

# Entrenar el nuevo pipeline
xgb_pipeline_monotonic.fit(X_train, y_train)

# Realizar predicciones en el conjunto de validación
xgb_predictions_val_monotonic = xgb_pipeline_monotonic.predict(X_val)

#métrica
xgb_mae_val_monotonic = mean_absolute_error(y_val, xgb_predictions_val_monotonic)
print("Mean Absolute Error en set de validacion para XGBRegressor con relación monótona:", xgb_mae_val_monotonic)

# Guardar el modelo en un archivo .pkl
joblib.dump(xgb_pipeline_monotonic, 'xgb_regressor_monotonic_model.pkl')


['city', 'pop', 'shop', 'brand', 'capacity', 'price', 'day', 'month', 'year']
Mean Absolute Error en set de validacion para XGBRegressor con relación monótona: 2555.675522873293


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


['xgb_regressor_monotonic_model.pkl']

## 1.3 Optimización de Hiperparámetros con Optuna (2.0 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se le pide:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)
- Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados?
- Fije el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

In [ ]:
# Fijar semilla para reproducibilidad
random_state = 42

# Función objetivo para Optuna
def objective(trial):
    # Hiperparámetros para XGBRegressor
    xgb_params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'max_leaves': trial.suggest_int('max_leaves', 0, 100),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1),
        'random_state': random_state  # Fijar semilla aquí
    }

    # Hiperparámetro para OneHotEncoder
    min_frequency = trial.suggest_float('min_frequency', 0.0, 1.0)

    # Crear el transformador OneHotEncoder con min_frequency
    categorical_transformer = OneHotEncoder(drop='first', sparse=False, min_frequency=min_frequency)

    # Preprocesador
    preprocessor = ColumnTransformer(
        transformers=[
            ('numeric', numeric_transformer, ['pop', 'price']),
            ('categorical', categorical_transformer, ['year', 'day', 'month', 'city', 'shop', 'brand', 'capacity'])
        ],
        remainder='passthrough'
    )

    # Crear el pipeline
    model = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', XGBRegressor(**xgb_params))
    ])

    # Evaluar el modelo usando cross-validation
    mae = -cross_val_score(model, X_train, y_train, n_jobs=-1, scoring='neg_mean_absolute_error').mean()
    return mae

# Crear un estudio de Optuna con TPESampler y semilla fijada
sampler = TPESampler(seed=random_state)  # Fijar semilla en el sampler
study = optuna.create_study(direction='minimize', sampler=sampler)
study.optimize(objective, timeout=300) # 5 minutos de tiempo límite

# Resultados
print("Número de trials:", len(study.trials))
print("Mejor MAE:", study.best_value)
print("Mejores hiperparámetros:", study.best_params)

# Ajustar el mejor modelo usando el pipeline

best_params = study.best_params
best_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(**best_params, random_state=random_state))
])
best_model.fit(X_train, y_train)

# Guardar el mejor modelo
joblib.dump(best_model, 'xgb_optimized_model.pkl')


Número de trials: 134
Mejor MAE: 2413.5219286677457
Mejores hiperparámetros: {'learning_rate': 0.08857059201269463, 'n_estimators': 934, 'max_depth': 7, 'max_leaves': 59, 'min_child_weight': 2, 'reg_alpha': 0.18146223218306257, 'reg_lambda': 0.07764651665753064, 'min_frequency': 0.07568783545179171}


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [02:28:07] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "min_frequency" } are not used.

  warnings.warn(smsg, UserWarning)


['xgb_optimized_model.pkl']

## 1.4 Optimización de Hiperparámetros con Optuna y Prunners (1.7)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento?
- Utilizar `optuna.integration.XGBoostPruningCallback` como método de **Prunning**
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

El "prunning" en el contexto de la optimización de hiperparámetros con herramientas como Optuna se refiere a la técnica de detener anticipadamente la evaluación de una prueba (trial) si se determina que es poco probable que supere a las mejores pruebas encontradas hasta el momento. El objetivo de esta técnica es mejorar la eficiencia de la optimización, reduciendo el tiempo y recursos gastados en pruebas que no son prometedoras.

In [ ]:
# Ajustar el preprocesador a los datos de entrenamiento y transformar los datos de entrenamiento
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_val_preprocessed = preprocessor.transform(X_val)

optuna.logging.set_verbosity(optuna.logging.WARNING)

# Fijar semilla para reproducibilidad
random_state = 42

# Función objetivo para Optuna con prunning
def objective(trial):
    # Hiperparámetros para XGBRegressor
    xgb_params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'max_leaves': trial.suggest_int('max_leaves', 0, 100),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1),
        'random_state': random_state
    }

    # Crear el transformador OneHotEncoder con min_frequency
    min_frequency = trial.suggest_float('min_frequency', 0.0, 1.0)
    categorical_transformer = OneHotEncoder(drop='first', sparse=False, min_frequency=min_frequency)


    # Crear DMatrices para XGBoost
    dtrain = xgb.DMatrix(X_train_preprocessed, label=y_train)
    dval = xgb.DMatrix(X_val_preprocessed, label=y_val)

    # Ajustar el nombre de la métrica
    eval_metric = "mae"
    xgb_params['eval_metric'] = eval_metric

    # Entrenamiento de XGBoost con prunning
    pruning_callback = XGBoostPruningCallback(trial, "validation-" + eval_metric)
    bst = xgb.train(xgb_params, dtrain, num_boost_round=1000, evals=[(dval, "validation")], early_stopping_rounds=10, callbacks=[pruning_callback], verbose_eval=False)

    # Predicciones y cálculo de MAE
    preds = bst.predict(dval)
    mae = mean_absolute_error(y_val, preds)
    return mae

# Crear un estudio de Optuna con prunning y semilla fijada
sampler = TPESampler(seed=random_state)
study = optuna.create_study(direction='minimize', sampler=sampler)
study.optimize(objective, timeout=300)  # 5 minutos de tiempo límite

# Resultados
print("Número de trials:", len(study.trials))
print("Mejor MAE:", study.best_value)
print("Mejores hiperparámetros:", study.best_params)

# Ajustar el mejor modelo usando los mejores hiperparámetros
best_params = study.best_params
best_model = xgb.XGBRegressor(**best_params, random_state=random_state)
best_model.fit(preprocessor.transform(X_train), y_train)

# Guardar el mejor modelo
joblib.dump(best_model, 'xgb_optimized_pruning_model.pkl')

Número de trials: 509
Mejor MAE: 1970.1523638768747
Mejores hiperparámetros: {'learning_rate': 0.09162027499379173, 'n_estimators': 610, 'max_depth': 8, 'max_leaves': 93, 'min_child_weight': 4, 'reg_alpha': 0.6617778340528291, 'reg_lambda': 0.7574868619920472, 'min_frequency': 0.8243313179079801}


['xgb_optimized_pruning_model.pkl']

## 1.5 Visualizaciones (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

- Gráfico de historial de optimización
- Gráfico de coordenadas paralelas
- Gráfico de importancia de hiperparámetros

Comente sus resultados: ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo?

In [ ]:
# Inserte su código acá

## 1.6 Síntesis de resultados (0.3)

Finalmente, genere una tabla resumen del MAE obtenido en los 5 modelos entrenados (desde Baseline hasta XGBoost con Constraints, Optuna y Prunning) y compare sus resultados. ¿Qué modelo obtiene el mejor rendimiento?

Por último, cargue el mejor modelo, prediga sobre el conjunto de test y reporte su MAE. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto?

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>